##Set up


In [129]:
# lots of imports for now. Will strip it down later

import sys
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import random
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, plot_roc_curve, roc_auc_score, plot_confusion_matrix
from sklearn.inspection import permutation_importance

from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
# import spotipy.util as util

#from google.colab import files
#from google.colab import drive
#drive.mount('drive', force_remount=True)

In [128]:
# data_path references data.csv from our github repo
data_path = 'https://raw.githubusercontent.com/StephenSpicer/Spotify_Music_Discovery_LS_DS_BW/main/data/data.csv'

## Wrangle Data


In [127]:
# cleaned wrangle functions:

def wrangle(filepath):
  df = pd.read_csv(filepath, parse_dates=['release_date'])
  
  #dropping name, artist column
  df = df.drop(['name', 'artists'], axis=1)         


  # set ID as the index
  df.set_index('id', inplace=True)
  
  # rearraging similar features to be together to improve readability
  df = df[['valence', 'acousticness', 'danceability',
        'energy', 'instrumentalness', 'liveness', 'loudness',  
        'speechiness','tempo'
        ]]

  # fixing loudness and tempo      
  df['loudness'] = df.loudness.apply(lambda x: x / -100)
  df['tempo'] = df.tempo.apply(lambda x: x / 1000)
  df = df[df.loudness > 0.0]

  return df

def the_key(filepath):
  df = pd.read_csv(filepath, parse_dates=['release_date'])

  # fixing artist column by stripping  [   ]  '  and  "
  df['artists'] = df['artists'].str.strip("[]")
  df['artists'] = df['artists'].str.strip("'")
  df['artists'] = df['artists'].str.strip('"')
  df['artists'] = df['artists'].str.lower()
  df['name'] = df['name'].str.lower()

  # set id as the index
  df.set_index('id', inplace=True)

  # made changes down here (filipe)
  # combined song name and artist
  df['name_artist'] = df['name'] + ' - ' + df['artists']
  
  key = df['name_artist']

  return key


  

In [ ]:
key = the_key(data_path)
key.head()

In [ ]:
# i used name as the target
df = wrangle(data_path)
x = df
x.head()

In [34]:
def create_fit_model(features_df):
  model = NearestNeighbors(n_neighbors=10,
                         n_jobs = -1)
  knn_spotify = model.fit(x)
  return knn_spotify

In [125]:
# made changes here (filipe)
# set the function to only ask for user input, and we can give instructions
# on the webpage for the user to put in the name of the song and artist, 
# or any part of either.

def recommended_songs(user_input):
  key = the_key(data_path)
  # no need to in, then statement. Will just find what name_artist combo contains the user_input  
  selected_song = key.loc[key.str.contains(user_input)]

  #search the key df and return the song id
  song_id = selected_song.index.tolist()

  #feed the song id into the model
  song_row = x.loc[song_id, :]

  #model finds the NN and gives you back song id
  neigh_dist, neigh_index = knn_spotify.kneighbors(song_row)

  #random nn
  index = neigh_index.flat[0:10].tolist()
  # song_index = random.choice(index)

  #convert the index of the suggested song to the song_id
  #This code is returning the column headers, not the index atm
  # not sure how this code works, will talk to Nick.
  # song_suggest = key.iloc[song_index].to_frame().columns.item()

  # converting list to df for easier access
  recom_songs = key.iloc[index].to_frame()
   

  #return the song id to plug into the spotify appi and have song play
  return print(recom_songs['name_artist'])

In [ ]:
recommended_songs('highway to hell')

The above function is almost done, here are the problems that need to be fixed:

1. the song_suggest var isn't grabbing the index value, which is what I wanted, it's grabbing the column titles. 
2. the "if title and artist" doesn't work atm because it doens't know what to do w/ title and artist. This shouldn't be far off, but it's 4am and i need to sleep. 

This spotify app is just a starting point, i think song_id_uri should read as 'artist:song:2l4Nmf7uR3wl4FZ6rre0Pp' for example. 

the token and sp should get the user to pull up their spotify app in window. That would be sick. the results i pulled off a random page. Basically this code may or may not be relevant. 

##Spotify API stuff


In [ ]:
client_id = 'id'
client_secret = 'secret'

scope = 'user-library-read'
if len(sys.argv) > 1:
  username = sys.argv[1]
  else:
    print("Usage: %s username" % (sys.argv[0]))
    sys.exit()

In [ ]:
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
token = util.prompt_for_user_token(scope, client_id= client_id, client_secret = client_secret, redirect_uri='http://localhost:8881/')
sp = spotipy.Spotify(auth=token)

In [ ]:
#importing images for songs from api
  id_name = {}
  list_photo = {}
  for i in sp.current_user_playlists()['items']:
    id_name[i['name']] = i['uri'].split(':')[2]
    list_photo[i['uri'].split(':')[2]] = i['images'][0]['url']
  #the app below needs to be tied into the spotify app above
  def vis_songs(something):
    temp = df['url'].values
    plt.figure(figsize=(15,int(0.625 * len(temp))))
    columns = 5
    for i, url in enumerate(temp):
      plt.subplot(len(temp) / columns + 1, columns, i +1)
      image = io.imread(url)
      plt.imshow(image)
      plt.xticks(color = 'w', fontsize = 0.1)
      plt.yticks(color = 'w', fontsize = 0.1)
      plt.xlabel(df['name'].values[i], fontsize = 12)
      plt.tight_layout(h_pad=0.4, w_pad=0)
      plt.subplots_adjust(wspace=None, hspace=None)

    return plt.show()

In [ ]:
# Needs work. Theoretically will take spotify song ID from id_grab and play song through spotify platform

def spotify(song):
  song_suggest = song
  song_id_uri = spotify:artist:song_suggest
  token = util.prompt_for_user_token(scope, client_id= client_id, client_secret = client_secret, redirect_uri='http://localhost:8881/')
  sp = spotipy.Spotify(auth=token)
  results = spotify.song(song_id_uri)
  while results['next']:
    results = spotify.next(results)
  
